In [1]:
import Ipynb_importer
import cv2 as cv
import sys,os
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from numba import jit
from image_preprocessing import cdi_process,calc_entropy
from sklearn.preprocessing import StandardScaler,MinMaxScaler

%matplotlib inline
sns.set_style('darkgrid')

Home_os='C:/Users/jingy/Jupyter/Crime/Fire/Dataset/video/'
negative_os=Home_os+'negative/'
positive_os=Home_os+'positive/'

importing Jupyter notebook from C:\Users\jingy\Jupyter\Crime\Fire\Entropy_Based\Preprocessing\image_preprocessing\cdi_process.ipynb
importing Jupyter notebook from C:\Users\jingy\Jupyter\Crime\Fire\Entropy_Based\Preprocessing\image_preprocessing\calc_entropy.ipynb


In [2]:
video=os.listdir(positive_os)

for i in video:
    video_name=positive_os+i
    #print(video_name)
    cap = cv.VideoCapture(video_name)
    num=cap.get(7)
    if num!=30:
        print(i)
    while 1:
        ret,frame=cap.read() 
        if ret:
            cv.imshow('frame',frame)
            k = cv.waitKey(1) 
            if k ==27:     
                break 
        else:
            break
cv.destroyAllWindows() 

In [3]:
video=os.listdir(negative_os)
Entropy_list=[]
Cross_list=[]
for i in video:
    start=0
    video_name=negative_os+i
    cap = cv.VideoCapture(video_name)
    in_list1=[]
    in_list2=[]
    while 1:
        if start<=28:
            cap.set(cv.CAP_PROP_POS_FRAMES, start)
            _,frame=cap.read()
            _,nextframe=cap.read()

            frame = cv.resize(frame, (224, 224),interpolation=cv.INTER_CUBIC)
            nextframe = cv.resize(nextframe, (224, 224),interpolation=cv.INTER_CUBIC)
            gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            nextgray=cv.cvtColor(nextframe,cv.COLOR_BGR2GRAY)
            H=calc_entropy.entropy(gray)
            CROSS = calc_entropy.cross_entropy(gray,nextgray)
            in_list1.append(H)
            in_list2.append(CROSS)
            cv.imshow('frame',frame)
            
            k = cv.waitKey(1) 
            start+=1
            if k ==27:     
                break 
        else:
            break
    Entropy_list.append(in_list1)
    Cross_list.append(in_list2)
    cap.release()
cv.destroyAllWindows() 

In [4]:
video=os.listdir(positive_os)
P_Entropy_list=[]
P_Cross_list=[]
for i in video:
    start=0
    video_name=positive_os+i
    cap = cv.VideoCapture(video_name)
    in_list1=[]
    in_list2=[]
    while 1:
        if start<=28:
            cap.set(cv.CAP_PROP_POS_FRAMES, start)
            _,frame=cap.read()
            _,nextframe=cap.read()

            frame = cv.resize(frame, (224, 224),interpolation=cv.INTER_CUBIC)
            nextframe = cv.resize(nextframe, (224, 224),interpolation=cv.INTER_CUBIC)
            gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            nextgray=cv.cvtColor(nextframe,cv.COLOR_BGR2GRAY)
            H=calc_entropy.entropy(gray)
            CROSS = calc_entropy.cross_entropy(gray,nextgray)
            in_list1.append(H)
            in_list2.append(CROSS)
            cv.imshow('frame',frame)
            
            k = cv.waitKey(1) 
            start+=1
            if k ==27:     
                break 
        else:
            break
    P_Entropy_list.append(in_list1)
    P_Cross_list.append(in_list2)
    cap.release()
cv.destroyAllWindows() 

In [11]:

combine_entropy=Entropy_list+P_Entropy_list

combine_cross=Cross_list+P_Cross_list

min_max_scaler =MinMaxScaler()
for i in range(len(combine_entropy)):
    combine_entropy[i] = ((min_max_scaler.fit_transform(np.array(combine_entropy[i]).reshape(-1,1)))).reshape(-1).tolist()
    combine_cross[i] = ((min_max_scaler.fit_transform(np.array(combine_cross[i]).reshape(-1,1)))).reshape(-1).tolist() 
new_list1=[]
for i in combine_entropy:
    str1 = ' '.join([str(x) for x in i])
    new_list1.append(str1)

new_list2=[]
for i in combine_cross:
    str1 = ' '.join([str(x) for x in i])
    new_list2.append(str1)
print(len(new_list1))

268


In [12]:
x = [0] * 126
y =[1]*142
z=x+y

In [13]:
#combine_dict={'entropy':new_list,'label':z}
combine_dict1={'entropy':new_list1,'label':z}
df=pd.DataFrame(combine_dict1)

display(df)

y=df['label'].values

,entropy,label
0,1.0 0.7265773184180091 0.5228672243368919 0.43...,0
1,0.8121734809901398 0.7145043251047412 0.649296...,0
2,0.18218290479597954 0.0 0.09517517217090443 0....,0
3,1.0 0.94487591227454 0.8987255970334331 0.7661...,0
4,0.028809765711088176 0.0729515095657085 0.0624...,0
...,...,...
263,0.9599895913787106 0.9468325118233505 0.856802...,1
264,0.9182360818316724 0.9180089681648029 0.918538...,1
265,0.9964333291336054 1.0 0.9989896907067468 0.99...,1
266,0.97281451729918 0.9545911666478784 0.91106702...,1


In [14]:
data_df = df['entropy'].str.split(' ', expand=True)

data_df['Class'] =y
data_df.to_csv('C:/Users/jingy/Jupyter/Crime/Fire/Entropy_Based/CSV/Gray_entropy.csv')
display(data_df)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,Class
0,1.0,0.7265773184180091,0.5228672243368919,0.4306419279439808,0.40404196572040973,0.3793389641645888,0.38913515515039876,0.4058614452277709,0.41386017388924756,0.3647986768244209,...,0.008711799683879917,0.10117481190499689,0.0408101882035794,0.036437361794298795,0.1267520865219467,0.15972702645331083,0.1275555047066348,0.0,0.008685113079366147,0
1,0.8121734809901398,0.7145043251047412,0.6492961845901277,0.7724619470509708,0.7637404538276087,0.7568534147249864,0.7754057647345292,0.7326782239205443,0.7992643381286371,0.8001548357232195,...,0.6663025440600734,0.7886948318952705,0.7879757523731996,1.0,0.2735758387325973,0.07945423147268116,0.034563914643257476,0.0,0.039189344921368274,0
2,0.18218290479597954,0.0,0.09517517217090443,0.1550342203235715,0.11400078729280949,0.04132073376651846,0.161995034960988,0.15309035206507104,0.07177296253044574,0.1463192145408314,...,0.6790644294700314,0.700861008893142,1.0,0.774371316795893,0.7406273027357457,0.9415279284684459,0.904296545850201,0.7531919772876066,0.5155082159806881,0
3,1.0,0.94487591227454,0.8987255970334331,0.7661158947832263,0.5338644000321295,0.38087882425546127,0.38637060813346125,0.2402696127913373,0.369816054879081,0.23623653357165608,...,0.55436879687106,0.5547007143712648,0.55436879687106,0.5547007143712648,0.5965004428903171,0.5876145860811732,0.5922141817457316,0.5909895299745926,0.5921229394142244,0
4,0.028809765711088176,0.0729515095657085,0.062468142863245646,0.0,0.004040657326161323,0.03735769362952013,0.08574247339549146,0.10731931173439335,0.14097182420336907,0.3771096954047053,...,0.9711562945210019,1.0,0.8908881119540872,0.8312214851610875,0.8369585989902895,0.8589238592580095,0.9038373670683626,0.9406150347164086,0.9323612425455998,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,0.9599895913787106,0.9468325118233505,0.8568027619980434,0.9066095738433866,0.9136364219109483,0.704435304606915,0.8451906065276935,0.6576404556597994,0.6474971269567611,1.0,...,0.2614015878929621,0.38455780783604077,0.4217390918483215,0.2946698452757506,0.31563179238751005,0.2941134320979746,0.4429761170995521,0.3424244644683121,0.38883033260737676,1
264,0.9182360818316724,0.9180089681648029,0.9185383763997077,0.920121174506793,0.9214820875196379,0.9676958218599356,0.713886234896222,0.702931579487915,0.31843556846911714,0.12072946027283304,...,0.7262056594865962,0.6486859301895018,0.5937223898035651,0.5652960563752378,0.5907438161377785,0.4801105454777934,0.16908192918432174,0.005046652673095675,0.0,1
265,0.9964333291336054,1.0,0.9989896907067468,0.9982582384053984,0.8348852161359019,0.6531064246740286,0.43408696731592755,0.5112467393557196,0.588741571687649,0.6016075759523183,...,0.2667153013480963,0.2775043565317148,0.202072978107811,0.23661758917230102,0.16496718347988448,0.09227137267706809,0.34169927754759755,0.1971214630420164,0.0,1
266,0.97281451729918,0.9545911666478784,0.9110670259173457,0.6577461670220792,0.7787548164981786,0.9519959400182705,0.892214793439364,0.8759419812708105,0.8203685805538123,0.9900844580072121,...,0.0,0.7219118800487774,0.4887397135125404,0.7604104337655135,0.728668771909728,0.7458379031810782,0.8546180714359286,0.7786023244345364,0.9712057846286131,1


In [15]:
combine_dict2={'cross':new_list2,'label':z}
df=pd.DataFrame(combine_dict2)

display(df)

y=df['label'].values

,cross,label
0,1.0 0.7181465445341075 0.5461938440106451 0.42...,0
1,0.7586160454472974 0.631535938682191 0.5459914...,0
2,0.1945569011490349 0.0 0.06532099813760794 0.1...,0
3,0.7890612278162337 0.7831618867028283 0.714836...,0
4,0.07477941444045655 0.053182836788547405 0.063...,0
...,...,...
263,0.6529862512526092 0.6800730092821814 0.582457...,1
264,0.6828021021561255 0.6829047693656212 0.682838...,1
265,0.2927361740031307 0.29388021510003437 0.29330...,1
266,0.16554643640413147 0.11390667118253317 0.2284...,1


In [16]:
data_df = df['cross'].str.split(' ', expand=True)

data_df['Class'] =y
data_df.to_csv('C:/Users/jingy/Jupyter/Crime/Fire/Entropy_Based/CSV/Gray_cross.csv')
display(data_df)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,Class
0,1.0,0.7181465445341075,0.5461938440106451,0.4234494902394772,0.4007126396416254,0.3714905415121166,0.3482395900001052,0.38944561606845696,0.40725183208525095,0.34243044680818,...,0.024042391499449423,0.10222608287392632,0.02654345984325346,0.1382104716633421,0.10111398561650731,0.13333143693239435,0.09260354659821246,0.0,0.0032145481626173478,0
1,0.7586160454472974,0.631535938682191,0.5459914690361813,0.6663706022649762,0.6715672697955881,0.6467963005694912,0.6713505090082208,0.6216721064084254,0.6593778001107466,0.6894024675050332,...,0.5919773415643306,0.6948693693998393,0.6767303207456052,1.0,0.2635250969902643,0.09038999504247158,0.03373493188766474,0.0,0.04283431096132517,0
2,0.1945569011490349,0.0,0.06532099813760794,0.1474183670516993,0.1350104236360039,0.05856221104666304,0.13836555187683786,0.15259393988390002,0.030047289942153554,0.2515182682478212,...,0.6442050996997182,0.7573064862616121,1.0,0.7098020878094076,0.6958781904434801,0.8766653187232833,0.9037346049317492,0.7683719099114441,0.6230664530921146,0
3,0.7890612278162337,0.7831618867028283,0.7148361539242103,1.0,0.45312960243707323,0.3181583189835493,0.3180018337876618,0.3856397627612864,0.2949499149898287,0.3442239777673848,...,0.37873640098406725,0.37899221655079884,0.37873640098406725,0.5338757156947622,0.41226390196470675,0.4051167885954925,0.4080318544380077,0.4070982640567564,0.40798004411954025,0
4,0.07477941444045655,0.053182836788547405,0.06305405334262204,0.0,0.006541813564261645,0.03839127295162825,0.06810970040601916,0.09635065272642507,0.38237869287608817,0.31739401556037095,...,0.8820034264975263,0.9641076687466565,0.8415200131892924,0.7126033201735318,0.7019542103374015,0.7594334598829136,0.7618623056803919,0.7887253474695619,0.7852723831988406,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,0.6529862512526092,0.6800730092821814,0.5824579523613505,0.6409290310116127,0.6901831756397003,0.5020442085902292,0.685582656634943,0.43522397107233735,0.5297294123457039,0.9211580632956027,...,0.17824608131464847,0.24151911259981862,0.27028999779005547,0.18096115935960544,0.19095324268191272,0.1957886262625883,0.286873132079708,0.2088572812697551,0.24086748094489607,1
264,0.6828021021561255,0.6829047693656212,0.6828387905335491,0.683911101867829,0.8298942762188721,1.0,0.5490954306962883,0.6670262783262646,0.2744780623626224,0.09160937955112702,...,0.5589403260648389,0.4883096205475117,0.44871660069917096,0.43333490544556685,0.45664712537957053,0.38149262793451033,0.14066593672670136,0.0,0.00035993170338777247,1
265,0.2927361740031307,0.29388021510003437,0.29330654087307284,0.3306922445737337,0.36090893518895495,0.2367576136394307,0.24702503449353497,0.1567215304795342,0.1942707772497867,0.1627287283858445,...,0.032825437275715075,0.02914410418353164,0.0,0.022980026325955194,0.011239048253883333,0.5452059997135263,0.9999999999999991,0.7879011139373917,0.6165533706970381,1
266,0.16554643640413147,0.11390667118253317,0.228467996864127,0.00973938460380186,0.9101651408547164,0.32894522590924735,0.08410409919535544,0.09466447260874222,0.13223029412217002,0.29392664358784426,...,0.01174174284514562,0.05644866491700684,0.0626741454006936,0.030228204887124743,0.0269454281080721,0.4603390213451597,0.21911837336300755,0.257725632092936,0.13535904805536614,1
